# 주어진 문장을 단어로 토큰화

In [21]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

text = '해보지 않으면 해낼 수 없다' # 토큰화할 문장

result = text_to_word_sequence(text) # 문장 토큰화
print(f'원문 : {text} \n토큰화된 문장 : {result}') # 결과 출력

원문 : 해보지 않으면 해낼 수 없다 
토큰화된 문장 : ['해보지', '않으면', '해낼', '수', '없다']


In [22]:
from keras.preprocessing.text import Tokenizer

docs = ['먼저 텍스트의 각 단어를 나누어 토큰화 합니다.',
        '텍스트의 단어로 토큰화 해야 딥러닝에서 인식됩니다.',
        '토큰화 한 결과는 딥러닝에서 사용할 수 있습니다.']  # 토큰화할 문장

token = Tokenizer() # 토큰화 객체 생성
token.fit_on_texts(docs) # 문장 토큰화

print(f'단어 카운트 : {dict(token.word_counts)},{type(token.word_counts)}\n') # 전체 문서에서 각 단어의 등장 횟수
print(f'문장 카운트 : {token.document_count}\n') # 문서 내 문장의 개수
print(f'각 단어가 몇 개의 문장에 포함되어 있는가 : {dict(token.word_docs)},{type(token.word_docs)}\n') # 각 단어마다 몇 개의 문장에서 등장하는지
print(f'각 단어에 매겨진 인덱스 값 : {dict(token.word_index)},{type(token.word_index)}') # 단어마다 부여된 인덱스 번호

단어 카운트 : {'먼저': 1, '텍스트의': 2, '각': 1, '단어를': 1, '나누어': 1, '토큰화': 3, '합니다': 1, '단어로': 1, '해야': 1, '딥러닝에서': 2, '인식됩니다': 1, '한': 1, '결과는': 1, '사용할': 1, '수': 1, '있습니다': 1},<class 'collections.OrderedDict'>

문장 카운트 : 3

각 단어가 몇 개의 문장에 포함되어 있는가 : {'단어를': 1, '먼저': 1, '합니다': 1, '각': 1, '텍스트의': 2, '나누어': 1, '토큰화': 3, '해야': 1, '딥러닝에서': 2, '인식됩니다': 1, '단어로': 1, '결과는': 1, '사용할': 1, '있습니다': 1, '수': 1, '한': 1},<class 'collections.defaultdict'>

각 단어에 매겨진 인덱스 값 : {'토큰화': 1, '텍스트의': 2, '딥러닝에서': 3, '먼저': 4, '각': 5, '단어를': 6, '나누어': 7, '합니다': 8, '단어로': 9, '해야': 10, '인식됩니다': 11, '한': 12, '결과는': 13, '사용할': 14, '수': 15, '있습니다': 16},<class 'dict'>


# 단어의 원핫인코딩

In [24]:
text = ['오랫동안 꿈꾸는 이는 그 꿈을 닮아간다']

token = Tokenizer()
token.fit_on_texts(text)

print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [26]:
x = token.texts_to_sequences(text) # 각 토큰의 인덱스
print(x)

[[1, 2, 3, 4, 5, 6]]


In [28]:
from tensorflow.keras.utils import to_categorical

word_size = len(token.word_index)+1 # 각 배열 가장 앞에 0이 추가되므로 +1(파이썬의 인덱스는 0부터 시작하지만, 현재 토큰은 1부터 시작이기 때문)
x = to_categorical(x, # 원핫인코딩을 진행할 데이터 
                   num_classes=word_size) # 클래스의 개수

print(x)

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


# 영화 리뷰의 긍정/부정 예측

In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding

In [33]:
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다","한번 더 보고싶네요",
        "글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]

classes = np.array([1,1,1,1,1,0,0,0,0,0]) # 긍정=1, 부정=0 클래스 지정

token = Tokenizer()
token.fit_on_texts(docs)

print(f'각 단어에 매겨진 인덱스 값 : \n{token.word_index}')

각 단어에 매겨진 인덱스 값 : 
{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [34]:
x = token.texts_to_sequences(docs)
print('토큰화 결과:\n',  x)

토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [35]:
# 모델이 입력할 데이터로 사용하기 위해 패딩(데이터의 길이를 동일하게 맞춤)
padded_x = pad_sequences(x, # 사용할 데이터
                         4) # 맞춰줄 길이
print('패딩 결과:\n', padded_x)

패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [40]:
# 임베딩
# 단순한 원핫 인코딩은 결과가 sparse한 형태가 됨
# 이를 해결하기 위해 단어의 의미를 고려하여 좀 더 조밀한 차원에 단어를 벡터로 표현하는 것

# 임베딩에 입력될 단어 수 지정
word_size = len(token.word_index) +1 # 파이썬의 인덱스는 0부터 시작

In [37]:
# 임베딩을 포함하는 모델 생성

model = Sequential()
model.add(Embedding(word_size, # 입력되는 단어 개수(단어 집합의 단어 개수)
                    8, # 출력값의 개수
                    input_length=4)) # 입력되는 단어 길이, 패딩을 통해 4로 맞추어 주었으므로 4로 지정
model.add(Flatten()) # 1차원으로 변환
model.add(Dense(1, activation='sigmoid')) # 긍정/부정 이진 분류이므로 출력 1개, 활성화함수 시그모이드

In [38]:
# 모델 컴파일 및 학습
model.compile(loss = 'binary_crossentropy', # 이진분류이므로 오차함수는 이진교차엔트로피
              optimizer = 'adam', # 최적화함수는 adam
              metrics=['accuracy']) # 평가지표는 정확도

model.fit(padded_x, classes, epochs=20) # 패딩된 데이터와 미리 지정한 클래스로 학습, 반복횟수는 20

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.6880 - accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 17ms/step - loss: 0.6856 - accuracy: 0.6000
Epoch 3/20
1/1 [==============================] - 0s 21ms/step - loss: 0.6832 - accuracy: 0.7000
Epoch 4/20
1/1 [==============================] - 0s 13ms/step - loss: 0.6808 - accuracy: 0.7000
Epoch 5/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6784 - accuracy: 0.8000
Epoch 6/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6761 - accuracy: 0.8000
Epoch 7/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6737 - accuracy: 0.9000
Epoch 8/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6713 - accuracy: 0.9000
Epoch 9/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6689 - accuracy: 0.9000
Epoch 10/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6666 - accuracy: 0.9000
Epoch 11/20
1/1 [================

In [39]:
# 결과 출력
print(f'loss : {model.evaluate(padded_x, classes)[0]:.4f}\nAccuracy : {model.evaluate(padded_x, classes)[1]:.4f}')

1/1 [==============================] - 0s 23ms/step - loss: 0.6400 - accuracy: 0.9000
loss : 0.6400
Accuracy : 0.9000
